In [57]:
"""
Adapted from https://github.com/declassengine/fgv/blob/master/Notebooks/cables_analysis_nb.ipynb
"""
import os
import sys
import time
import datetime
import re
import pickle
import getpass
from bs4 import BeautifulSoup
import pandas as pd
import pandas.io.sql as psql
import pymysql
import numpy as np
import pylab
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import seaborn as sns
#import seaborn.apionly as sns
import nltk
import nltk.data
from nltk.corpus import stopwords
from collections import defaultdict

%matplotlib inline
#%pylab inline

In [58]:
path = "/Users/clarahsuong/Dropbox/nyu_postdoc/ner/dataset_intro/"

In [59]:
p = getpass.getpass()
conn = pymysql.connect(host='history-lab.org', port=3306, user='de_reader', passwd=p, db='declassification_cables')
cur = conn.cursor()

········


In [60]:
cur.execute("show databases;")
for r in cur.fetchall():
   print(r)

('information_schema',)
('authentication',)
('bookwormDB',)
('clinton_test',)
('clinton_test_2',)
('ddrs_equity',)
('declassification',)
('declassification_api',)
('declassification_api_test',)
('declassification_api_update',)
('declassification_cabinet',)
('declassification_cables',)
('declassification_clinton',)
('declassification_clinton_staging',)
('declassification_cpdoc',)
('declassification_ddrs',)
('declassification_foia_dod',)
('declassification_frus',)
('declassification_frus_update',)
('declassification_kissinger',)
('declassification_pdb',)
('declassification_pdb_test',)
('etc',)
('historylab_user_information',)
('mysql',)
('performance_schema',)
('predict_history',)
('predict_history_new',)
('predictify_source',)
('predictify_target',)
('sys',)
('user_information',)
('visualizations',)


In [61]:
cur.execute("use declassification_cables;")
cur.execute("show tables;")
for r in cur.fetchall():
   print(r)

('classification_countries',)
('classification_doc',)
('classifications',)
('concept_doc',)
('concepts',)
('countries',)
('country_doc',)
('doc_counts',)
('docs',)
('from_to_sum',)
('network_docs',)
('network_nodes',)
('office_doc',)
('offices',)
('person_doc',)
('persons',)
('reference_doc',)
('tag_doc',)
('tag_doc_staging',)
('tagname_doc',)
('tagnames',)
('tags',)
('tags_staging',)
('tokens',)
('top_classifications',)
('top_countries',)
('top_network',)
('top_persons',)
('top_topics',)
('topic_doc',)
('topic_token',)
('topics',)
('urgency',)
('urgency_doc',)


In [62]:
cur.execute("desc declassification_cables.docs;")
print('Number of fields is {}'.format(cur.rowcount))
print
fields = []
for r in cur.fetchall():
    fields.append(r)
    print(r)

Number of fields is 80
('doc_nbr', 'varchar(30)', 'YES', '', None, '')
('auto_decaption', 'varchar(16)', 'YES', '', None, '')
('reference', 'text', 'YES', '', None, '')
('capture_date', 'date', 'YES', '', None, '')
('channel', 'varchar(32)', 'YES', '', None, '')
('concepts', 'text', 'YES', '', None, '')
('control_nbr', 'varchar(32)', 'YES', '', None, '')
('copy', 'varchar(32)', 'YES', '', None, '')
('date', 'date', 'YES', 'MUL', None, '')
('decaption_date', 'text', 'YES', '', None, '')
('decaption_note', 'varchar(255)', 'YES', '', None, '')
('disp_action', 'varchar(32)', 'YES', '', None, '')
('disp_approved_on_date', 'date', 'YES', '', None, '')
('disp_case', 'varchar(32)', 'YES', '', None, '')
('disp_comment', 'text', 'YES', '', None, '')
('disp_date', 'date', 'YES', '', None, '')
('disp_event', 'varchar(8)', 'YES', '', None, '')
('disp_history', 'text', 'YES', '', None, '')
('disp_reason', 'varchar(32)', 'YES', '', None, '')
('disp_remarks', 'varchar(8)', 'YES', '', None, '')
('doc_s

In [63]:
fields = list(zip(*fields))[0]
print(fields)

('doc_nbr', 'auto_decaption', 'reference', 'capture_date', 'channel', 'concepts', 'control_nbr', 'copy', 'date', 'decaption_date', 'decaption_note', 'disp_action', 'disp_approved_on_date', 'disp_case', 'disp_comment', 'disp_date', 'disp_event', 'disp_history', 'disp_reason', 'disp_remarks', 'doc_source', 'drafter', 'enclosure', 'eo', 'errors', 'expiration', 'film', 'handling', 'isecure', 'legacy_key', 'line_count', 'litigationhistory', 'locator', 'messageid', 'office', 'origclass', 'orighand', 'origpclass', 'origphand', 'page_count', 'pchannel', 'pclass', 'phandling', 'retention', 'review_action', 'review_content_flags', 'review_date', 'review_event', 'review_exemptions', 'review_media_id', 'review_release_date', 'review_release_event', 'review_transfer_date', 'review_withdrawn_fields', 'review_markings', 'sasid', 'secure', 'status', 'subject', 'to_field', 'vdkvgwkey', 'markings', 'body', 'raw_body', 'nara_markings', 'type', 'format', 'from_field', 'class', 'id', 'cable_type', 'source_

In [64]:
print('Uncomment to run - otherwise load pickle next cell')

distinct_fields_all = defaultdict(list)
for field in fields:
    query = """
    SELECT  docs.{0}, COUNT(docs.{0}) AS qty
    FROM     declassification_cables.docs
    GROUP BY docs.{0}
    ORDER BY qty DESC
    LIMIT    30;
    """.format(field)
    cur.execute(query)
    #print('Frequent values for the field {} (max 30):'.format(field))
    for r in cur.fetchall():
        #print(r)
        distinct_fields_all[field].append(r)
    #print
    #print
with open(os.path.join(path, 'distinct_fields_all.pkl'), 'wb') as f:
  pickle.dump(distinct_fields_all, f)


Uncomment to run - otherwise load pickle next cell


OperationalError: (2013, 'Lost connection to MySQL server during query ([Errno 54] Connection reset by peer)')

##Word frequency among the documents
from nltk import ngrams, FreqDist
all_counts = dict()
for size in 2, 3, 4, 5:
    all_counts[size] = FreqDist(ngrams(data, size))
    

collections.Counter(bigrams) or pandas.Series(bigrams).value_counts()    

##Sentiment analysis of body of full-text cables 

##Length of documents
